In [8]:
import pandas as pd
import numpy as np

In [44]:
RACE_COLS = ['pctwhite', 'pctblack', 'pctapi', 'pctaian', 'pct2prace', 'pcthispanic']

In [10]:
!ls ../data

WikiLabels.csv  census_2000.csv census_2010.csv


In [16]:
census_2000 = pd.read_csv('../data/census_2000.csv')
census_2010 = pd.read_csv('../data/census_2010.csv')

In [47]:
census = census_2000.append(census_2010, ignore_index=True)

In [49]:
census.sample(10)

,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
229413,VANDERWYST,77522,247,0.08,85661.83,99.6,0,(S),0,0,(S)
154254,BEAM,2584,13972,4.74,52461.61,92.66,3.06,0.4,0.46,1.47,1.95
74098,PORSCH,73931,244,0.09,85354.89,93.44,999,999,0,0,4.51
301707,KIRSHY,149395,110,0.04,89630.15,100,0,0,0,0,0
222479,ARMANTROUT,70645,277,0.09,85047.99,93.14,0,(S),2.89,(S),2.53
233831,PYTKO,82044,230,0.08,86018.63,98.7,(S),0,0,(S),0
283816,BEERENS,131379,129,0.04,88909.74,97.67,0,0,0,(S),(S)
237113,INGWERSON,85357,219,0.07,86268.54,89.5,0,4.11,(S),3.2,(S)
172013,ROYALS,20336,1307,0.44,75531.46,76.59,18.97,(S),(S),1.53,2.45
113952,PIAO,113519,143,0.05,88090.05,999,999,95.1,0,999,999


In [45]:
census.apply(lambda x: x == '(S)').sum()

name                0
rank                0
count               0
prop100k            0
cum_prop100k        0
pctwhite         5075
pctblack        68501
pctapi          73062
pctaian         53615
pct2prace       63138
pcthispanic     51950
dtype: int64

In [51]:
for r in RACE_COLS:
    census[r] = census[r].apply(lambda x: 999 if x == '(S)' else float(x), 1)

In [54]:
census.tail()

,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
313920,DIETZMANN,160975,100,0.03,90062.93,96.00,0.00,0.00,999.00,0.00,999.00
313921,DOKAS,160975,100,0.03,90062.96,94.00,999.00,0.00,0.00,999.00,999.00
313922,DONLEA,160975,100,0.03,90062.99,94.00,0.00,0.00,0.00,0.00,6.00
313923,DORIOTT,160975,100,0.03,90063.03,89.00,0.00,999.00,0.00,5.00,999.00
313924,ALL OTHER NAMES,0,29312001,9936.97,9936.97,66.65,8.53,7.97,0.86,2.32,13.67


In [71]:
%%time
for idx, row in census.iterrows():
    if idx % 10_000 == 0:
        print(f'{idx} row done')
    if sum(row[RACE_COLS]) > 100:
        missing_race = []
        missing_value = 100
        for k, v in row[RACE_COLS].items():
            if v > 100:
                missing_race.append(k)
            else:
                missing_value -= v

        if missing_race:
            missing_value /= len(missing_race)

            for race in missing_race:
                census.loc[idx, race] = missing_value

0 row done
10000 row done
20000 row done
30000 row done
40000 row done
50000 row done
60000 row done
70000 row done
80000 row done
90000 row done
100000 row done
110000 row done
120000 row done
130000 row done
140000 row done
150000 row done
160000 row done
170000 row done
180000 row done
190000 row done
200000 row done
210000 row done
220000 row done
230000 row done
240000 row done
250000 row done
260000 row done
270000 row done
280000 row done
290000 row done
300000 row done
310000 row done
CPU times: user 18min 4s, sys: 6min 15s, total: 24min 20s
Wall time: 24min 25s


In [94]:
census.groupby('name').agg({
    'pctwhite': np.mean, 'pctblack': np.mean, 'pctapi': np.mean, 'pctaian': np.mean, 'pct2prace': np.mean, 'pcthispanic': np.mean, 'count': max}).reset_index().to_csv('../data/census.csv', index=False)

In [95]:
census.name.nunique()

167408

In [99]:
census['count'].sum()/1e8

5.37100302